## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
llm_config = {"model": "gemini-1.5-flash","api_key":"putYourKey","api_type":"google"}

## Define an AutoGen agent

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [3]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

{'content': "Why don't scientists trust atoms? \n\nBecause they make up everything!\n", 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


In [4]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

{'content': "Please tell me the joke you'd like me to repeat! I need the joke's text to be able to repeat it.\n", 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [6]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

(Cathy adjusts the mic, a mischievous glint in her eye)  Alright Joe, alright everybody!  So, I was at the grocery store the other day, right? And I'm trying to find the organic kale – you know, the kind that costs more than my rent.  And this guy bumps into me, spills his entire basket of…wait for it… *bananas*.  I swear, I thought he was gonna apologize, but instead he just stares at the bananas for a solid minute, then says, "Well, they're already bruised."  I mean, *sir*, I'm pretty sure *that's* the point of organic produce!  It's supposed to be imperfectly perfect!  Or maybe *perfectly* imperfect?  I’m still working on the marketing angle.


--------------------------------------------------------------------------------
joe (to cathy):

(Joe takes the mic, a wry smile playing on his lips)  …I’m still working on the mark

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [7]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': '(Cathy adjusts the mic, a mischievous glint in her eye)  Alright '
             'Joe, alright everybody!  So, I was at the grocery store the '
             "other day, right? And I'm trying to find the organic kale – you "
             'know, the kind that costs more than my rent.  And this guy bumps '
             'into me, spills his entire basket of…wait for it… *bananas*.  I '
             'swear, I thought he was gonna apologize, but instead he just '
             'stares at the bananas for a solid minute, then says, "Well, '
             'they\'re already bruised."  I mean, *sir*, I\'m pretty sure '
             "*that's* the point of organic produce!  It's supposed to be "
             'imperfectly perfect!  Or maybe *perfectly* imperfect?  I’m still '
             'working on the marketing angle.\n',
  'name': 'cathy',
  'role': 'user'},
 {'content': '(Joe takes 

In [8]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'total_cost': 0},
 'usage_including_cached_inference': {'gemini-1.5-flash': {'completion_tokens': 410,
                                                           'cost': 0.00016312500000000002,
                                                           'prompt_tokens': 535,
                                                           'total_tokens': 945},
                                      'total_cost': 0.00016312500000000002}}


In [9]:
pprint.pprint(chat_result.summary)

("(Cathy bursts out laughing, then claps Joe on the back)  Oh, that's *good*, "
 "Joe! That's *really* good!  You know, I think the banana slicer is the "
 'problem.  It\'s too… *intense*.  It screams, "I\'m a man who takes his '
 'bananas *very* seriously."  Women are intimidated by serious banana slicing, '
 "Joe.  They're looking for a guy who can handle a little… *peel*.  Maybe just "
 'stick to witty banter, like you just did.  Unless... you *actually* have a '
 'banana slicer collection.  Then, I might be intrigued.  (Winks)\n')


## Get a better summary of the conversation

In [10]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

(Cathy adjusts the mic, a mischievous glint in her eye)  Alright Joe, alright everybody!  So, I was at the grocery store the other day, right? And I'm trying to find the organic kale – you know, the kind that costs more than my rent.  And this guy bumps into me, spills his entire basket of…wait for it… *bananas*.  I swear, I thought he was gonna apologize, but instead he just stares at the bananas for a solid minute, then says, "Well, they're already bruised."  I mean, *sir*, I'm pretty sure *that's* the point of organic produce!  It's supposed to be imperfectly perfect!  Or maybe *perfectly* imperfect?  I’m still working on the marketing angle.


--------------------------------------------------------------------------------
joe (to cathy):

(Joe takes the mic, a wry smile playing on his lips)  …I’m still working on the mark

In [11]:
pprint.pprint(chat_result.summary)

{'audio': None,
 'content': "Joe's dating life is unsuccessful, despite his humor and charm. "
            'Cathy suggests his serious approach to banana slicing may be '
            'intimidating to potential partners, advising him to focus on '
            'witty conversation instead.  A banana slicer collection, however, '
            'might be intriguing.\n',
 'function_call': None,
 'refusal': None,
 'role': 'assistant',
 'tool_calls': None}


## Chat Termination

Chat can be terminated using a termination conditions.

In [12]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [13]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Okay Joe,  you asked for it!  So, I was at the zoo the other day, right?  And there's this guy standing in front of the llama enclosure, just staring.  Staring intensely.  Finally, I go up to him and say, "Having a good look at the llamas?"  And he says, "Yeah, I'm trying to figure out which one's the accountant."  I swear, I almost choked on my pretzel.  Get it?  Because they all look the *same*!  ...  ...  Okay, another one.  Why don’t scientists trust atoms?  Because they make up everything!

...So, what's your favorite kind of joke?  I'm feeling pretty good tonight, ready for another set!


--------------------------------------------------------------------------------
joe (to cathy):

(Chuckles, wiping a pretend sweat from his brow)  Oh man, Cathy, you’re killing me! That llama accountant joke… brilliant!  I love the dad

In [14]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was the one about the accountant llama.  You know, the one where the guy is trying to figure out which llama is the accountant because they all look the same?


--------------------------------------------------------------------------------
cathy (to joe):

Oh yeah!  The llama accountant!  Glad you liked it.  I've been working on a whole set about animals in the workplace, actually.  It's surprisingly rich material.  I gotta go!


--------------------------------------------------------------------------------
